In [60]:
import socket
import pickle
import time
import threading
from rsa import RSA  
from Client import CAClient  
def test_message_exchange():
    # Assume the CA server is already running on 127.0.0.1:50051.
    ca_public_key = None  
    
    # Generate RSA keys for Client A and Client B.
    rsa_A = RSA(1024)
    public_key_A, private_key_A = rsa_A.generate_keys()

    rsa_B = RSA(1024)
    public_key_B, private_key_B = rsa_B.generate_keys()

    # Instantiate Client A and Client B.
    clientA = CAClient("127.0.0.1", 50051, "A", public_key_A, private_key_A, ca_public_key)
    clientB = CAClient("127.0.0.1", 50051, "B", public_key_B, private_key_B, ca_public_key)

    # Use the same RSA instance that generated the keys.
    clientA.rsa = rsa_A
    clientB.rsa = rsa_B

    # Register both clients with the CA.
    clientA.register()  
    clientB.register()  

    time.sleep(0.5)

    # Each client requests a certificate from the CA.
    clientA.request_certificate()
    clientB.request_certificate()
    time.sleep(0.5)

    # Each client retrieves its peer's certificate.
    cert_for_A = clientA.get_peer_certificate("B")
    cert_for_B = clientB.get_peer_certificate("A")

    # Validate the certificates by asking the CA server to verify them.
    if not cert_for_A or not clientA.validate_certificate(cert_for_A):
        print("Peer certificate validation failed for Client B.")
        return
    if not cert_for_B or not clientB.validate_certificate(cert_for_B):
        print("Peer certificate validation failed for Client A.")
        return

    # Extract the public keys from the verified certificates.
    peer_public_key_for_A = cert_for_A["public_key"]
    peer_public_key_for_B = cert_for_B["public_key"]

    print(f"Client A's public key: {public_key_A}")
    print(f"Client B's public key: {public_key_B}")

    # Simulate the exchange of three messages based on the verified certificates.
    for i in range(1, 4):
        message = f"Hello{i}"
        print(f"\nClient A sending: {message}")
        encrypted_message = clientA.send_message(peer_public_key_for_A, message)
        received_message = clientB.receive_message(encrypted_message)
        print(f"Client B received: {received_message}")

        ack_message = f"ACK{i}"
        print(f"Client B sending: {ack_message}")
        encrypted_ack = clientB.send_message(peer_public_key_for_B, ack_message)
        received_ack = clientA.receive_message(encrypted_ack)
        print(f"Client A received: {received_ack}")

if __name__ == "__main__":
    test_message_exchange()


Client A registration successful.
Client B registration successful.
Certificate issued for Client A: {'client_id': 'A', 'public_key': (47587985333725583188414815770834567001649122604260968486714076114562354356217372564569440388923930033669518527510456314402681626410768123084284313767538722035164791520729853965131250749637903923760560378084777430194542880556021410606383821535662333343195293788499265364930809123813003765701296602513241837831, 118511524742555063110333128251173624570838857929330652696135252984107173756421532154483969864437259365397926000343877120627931714900639437784984974058579538765749365130020362039114025310664424343343201655087701469966712796827594617253220669966407839689288532017924021489417188527322736864420335899147415761563), 'issue_time': 1743356678, 'duration': 600, 'signature': 'PoP5df7okkvu8TkOPE1pAUpOPhy4b4hCKbDq8jkmKCl4rgl0hChjVBmAWYgaCeiO+Fe5x7Lq8C1vBWz2RL0gn2o+qwrTnnc29cgb5FqTRvxt1NtybSwxAA9MBmXH4irwmT4HvG1SaPiBPTHMHU1AuporOISnCXvOSMd3qUNgdPs='}
Certificate